In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from IPython.display import Image, display
from langgraph.prebuilt import create_react_agent
from langchain import hub
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import MultionToolkit
from langchain_aws import ChatBedrockConverse
import os
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import WebBaseLoader

load_dotenv()

In [ ]:
AWS_ACCESS_KEY_ID = 
AWS_SECRET_ACCESS_KEY = 

In [ ]:
class State(TypedDict):
    url: str
    bio: str
    industry: str

In [ ]:
website = 

In [ ]:
class BioInfor(BaseModel):
    industry: str 
    about: str

In [ ]:
# this agent to generate company bio and recognise industry
prompt = """
"""
llm = ChatBedrockConverse(
    model_id="anthropic.claude-3-5-sonnet-latest-v1:0"
)
web_toolkit = MultionToolkit()
tools = web_toolkit.get_tools()
bio_agent = create_react_agent(
    model = llm,
    tools = tools,
    prompt = prompt
    
)
def bio_agent_node(state: State):
    bio_parser = PydanticOutputParser(pydantic_object=BioInfo)
    result = bio_agent.invoke({"messages":[
        {"role": "user", "content": state["url"]}
    ]})
    try:
        bio_data = bio_parser.parse(extract_bio_json(result))
        return {"bio": bio_data.about, "industry": bio_data.industry}



In [ ]:
def industry_news_node(state: State):
    prompt = """
    """
    search = DuckDuckGoSearchRun()
    results = search.invoke(prompt + state["industry"])
    return state

In [ ]:
from urllib.parse import quote

def usecase_node(state: State):
    path = quote(str(state.get('industry', '')).strip('/'))
    loader = WebBaseLoader(f"https://aws.amazon.com/{path}/")
    return state